# 基于模式匹配的对话机器人实现


### Pattern Match

机器能否实现对话，这个长久以来是衡量机器人是否具有智能的一个重要标志。 Alan Turing早在其文中就提出过一个测试机器智能程度的方法，该方法主要是考察人类是否能够通过对话内容区分对方是机器人还是真正的人类，如果人类无法区分，我们就称之为具有”智能“。而这个测试，后来被大家叫做”图灵测试“，之后也被翻拍成了一步著名电影，叫做《模拟游戏》。 



既然图灵当年以此作为机器是否具备智能的标志，这项任务肯定是复杂的。自从 1960s 开始，诸多科学家就希望从各个方面来解决这个问题，直到如今，都只能解决一部分问题。 目前对话机器人的建立方法有很多，今天的作业中，我们为大家提供一共快速的基于模板的对话机器人配置方式。

In [99]:
def is_varaible(pattern):
    return pattern.startswith('?') and all(s.isalpha() for s in pattern[1:])

In [100]:
def pattern_match(pattern, saying):
    if not pattern or not saying: return []

    if is_varaible(pattern[0]):
        return [(pattern[0], saying[0])] + pattern_match(pattern[1:], saying[1:])
    else:
        if pattern[0] != saying[0]:
            return []
        else:
            return pattern_match(pattern[1:], saying[1:])

In [101]:
pattern_match('?A + ?B = ?C'.split(), '3 + 2 = 5'.split())

[('?A', '3'), ('?B', '2'), ('?C', '5')]

In [102]:
def substitute(rule, parsed_rules):
    if not rule:
        return []

    return [parsed_rules.get(rule[0], rule[0])] + substitute(rule[1:], parsed_rules)

In [103]:
got_patterns = pattern_match('I want ?X'.split(), 'I want Iphone'.split())

' '.join(substitute('What if you mean if you got ?X'.split(), dict(got_patterns)))

'What if you mean if you got Iphone'

In [104]:
defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
}

In [105]:
import random


def get_response(saying, rules=defined_patterns):
    for pattern in rules:
        got_patterns = pattern_match(pattern.split(), saying.split())
        if got_patterns:
            return ' '.join(substitute(random.choice(rules[pattern]).split(), dict(got_patterns)))
    else:
        return "I don't konwn how to response you."

In [106]:
print(get_response('I need Iphone'))
print(get_response('My Mother told me something'))
print(get_response("It's fine day"))

Image you will get Iphone soon
How do you think about your Mother ?
I don't konwn how to response you.


## Segment Match

我们上边的这种形式，能够进行一些初级的对话了，但是我们的模式逐字逐句匹配的， "I need iPhone" 和 "I need ?X" 可以匹配，但是"I need an iPhone" 和 "I need ?X" 就不匹配了，那怎么办？ 

为了解决这个问题，我们可以新建一个变量类型 "?\*X", 这种类型多了一个星号(\*),表示匹配多个

In [107]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(s.isalpha() for s in pattern[2:])

In [108]:
is_pattern_segment('?*Hello')

True

In [557]:
fail = [True, None]

def is_match(rest, saying):
    if not rest or not saying:
        return True

    if not all(a.isalpha() for a in rest[0]):
        return True

    if rest[0] != saying[0]:
        return False

    return is_match(rest[1:], saying[1:])
    

def segment_match(pattern, saying):
    seg_pattern, rest = pattern[0], pattern[1:]
    seg_pattern = seg_pattern.replace('?*', '?')

    if not rest: return (seg_pattern, saying), len(saying)

    for i, token in enumerate(saying):
        if rest[0] == token and is_match(rest[1:], saying[(i+1):]):
            return (seg_pattern, saying[:i]), i

    return (seg_pattern, saying), len(saying)


def pattern_match_with_seg(pattern, saying):
    if not pattern or not saying: return []

    if is_varaible(pattern[0]):
        return [(pattern[0], saying[0])] + pattern_match_with_seg(pattern[1:] ,saying[1:])
    elif is_pattern_segment(pattern[0]):
        match, index = segment_match(pattern, saying)
        return  [match] + pattern_match_with_seg(pattern[1:], saying[index:])
    elif pattern[0] == saying[0]:
        return pattern_match_with_seg(pattern[1:], saying[1:])
    else:
        return [True, None]
    

In [115]:
pattern_match_with_seg('?*P is very good!'.split(), 'My dog is very good!'.split())

[('?P', ['My', 'dog'])]

In [111]:
response_pair = {
    'I need ?X': [
        "Why do you neeed ?X"
    ],
    "I dont like my ?X": ["What bad things did ?X do for you?"]
}

In [141]:
def pattern2dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}


In [145]:
got_patterns = pattern_match_with_seg('I need ?*X'.split(), 'I need an ipad and an macbookpro'.split())

print(' '.join(substitute("Why do you neeed ?X".split(), pattern2dict(got_patterns))))


print(' '.join(substitute('What bad things did ?X for you?'.split(), pattern2dict(
            pattern_match_with_seg('I dont like my ?X'.split(), 'I dont like my Boss'.split())))))

Why do you neeed an ipad and an macbookpro
What bad things did Boss for you?


In [147]:
# ("?*X hello ?*Y", "Hi, how do you do")

' '.join(substitute('Hi, how do you do'.split(), 
                    pattern2dict(pattern_match_with_seg('?*X hello ?*Y'.split(), 'hello Mike'))))

'Hi, how do you do'

## Assignment

### 问题1
编写一个程序, get_response(saying, response_rules)输入是一个字符串 + 我们定义的 rules，例如上边我们所写的 pattern， 输出是一个回答。

In [258]:
import random

defined_patterns = {
    "I need ?*x": ["Image you will get ?x soon", "Why do you need ?x ?"], 
    "My ?*X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"],
}


def get_response(saying, rules=defined_patterns):
    for pattern in rules:
        got_patterns = pattern_match_with_seg(pattern.split(), saying.split())
        print(got_patterns)
        if got_patterns:
            return ' '.join(substitute(random.choice(rules[pattern]).split(), pattern2dict(got_patterns)))
    else:
        return "I don't konwn how to response you."

In [259]:
get_response('I need an Ipad')

[('?x', ['an', 'Ipad'])]


'Why do you need an Ipad ?'

### 问题2
改写以上程序，将程序变成能够支持中文输入的模式。
*提示*: 你可以需用用到 jieba 分词

In [620]:
rule_responses = {
    '?*x hello ?*y': ['How do you do'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x电视很不错': ['嗯嗯，?x是狠不错', '?x我也很喜欢'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}

In [621]:
import os
import jieba
import random


In [640]:
fail = (True, None)


def is_match(rest, saying):
    if not rest or not saying:
        return True

    # 分割下一个pattern, ?
    if not all(a.isalpha() for a in rest[0]):
        return True

    if rest[0] != saying[0]:
        return False

    return is_match(rest[1:], saying[1:])
    

def segment_match(pattern, saying):
    seg_pattern, rest = pattern[0], pattern[1:]
    seg_pattern = seg_pattern.replace('?*', '?')

    if not rest: return (seg_pattern, saying), len(saying)

    for i, token in enumerate(saying):
        if rest[0] == token and is_match(rest[1:], saying[(i+1):]):
            return (seg_pattern, saying[:i]), i

    return (seg_pattern, saying), len(saying)


def pattern_match_with_seg(pattern, saying):
    if all([not pattern, not saying]): return []
    if any([not pattern, not saying]): return [fail]

    if is_varaible(pattern[0]):
        return [(pattern[0], saying[0])] + pattern_match_with_seg(pattern[1:] ,saying[1:])
    elif is_pattern_segment(pattern[0]):
        match, index = segment_match(pattern, saying)
        return  [match] + pattern_match_with_seg(pattern[1:], saying[index:])
    elif pattern[0] == saying[0]:
        return pattern_match_with_seg(pattern[1:], saying[1:])
    else:
        return [fail]


def is_pattern_match(got_patterns):
    if not got_patterns:
        return False

    for k, v in got_patterns:
        if k is True and v is None:
            return False

    return True


def merge_token(tokens):
    if len(tokens) < 2:
        return tokens

    if tokens[0] == '?' and tokens[1].isalpha():
        return [''.join(tokens[:2])] + merge_token(tokens[2:])

    if len(tokens) > 2 and  tokens[0] == '?' and tokens[1] == '*' and tokens[2].isalpha():
        return [''.join(tokens[:3])] + merge_token(tokens[3:])

    return [tokens[0]] + merge_token(tokens[1:])


def is_chinese(string):
    for ch in string:
        if ch > '\u4e00' and ch < '\u9fff':
            return True
    else:
        return False


def cut_chinese(string):
    tokens = list(jieba.cut(string))
    return merge_token(tokens)


def cut(string):
    if is_chinese(string):
        return cut_chinese(string)
    else:
        return string.split()
    


def get_response(saying, rules=rule_responses):
    for pattern in rules:
        got_patterns = pattern_match_with_seg(cut(pattern), cut(saying))

#         print(pattern, got_patterns)
        if is_pattern_match(got_patterns):
            tokens = substitute(cut(random.choice(rules[pattern])), pattern2dict(got_patterns))
            if is_chinese(saying):
                return ''.join(tokens)
            else:
                return ' '.join(tokens)
            
    else:
        return "I don't konwn how to response you."

In [632]:
print(get_response('Mike hello John'))
print(get_response('Mike John I want an Ipad'))
print(get_response('亮剑电视很不错'))
print(get_response('你好'))

How do you do
Why do you want an Ipad
嗯嗯，亮剑是狠不错
我不太确定我很理解你说的, 能稍微详细解释一下吗?


In [636]:
import jieba


def is_chinese(string):
    for ch in string:
        if ch > '\u4e00' and ch < '\u9fff':
            return True
    else:
        return False


def merge_token(tokens):
    if len(tokens) < 2:
        return tokens

    if tokens[0] == '?' and tokens[1].isalpha():
        return [''.join(tokens[:2])] + merge_token(tokens[2:])

    if len(tokens) > 2 and  tokens[0] == '?' and tokens[1] == '*' and tokens[2].isalpha():
        return [''.join(tokens[:3])] + merge_token(tokens[3:])

    return [tokens[0]] + merge_token(tokens[1:])

    
def cut_chinese(string):
    tokens = list(jieba.cut(string))
    return merge_token(tokens)


def cut(string):
    if is_chinese(string):
        return cut_chinese(string)
    else:
        return string.split()

In [635]:
cut('?*x所有人?y')

['?*x', '所有人', '?y']

###  问题3

多设计一些模式，让这个程序变得更好玩，多和大家交流，看看大家有什么好玩的模式

TODO

### 问题4

1. 程序的优点，缺点以及改进的地方?

优点：程序实现基本的人机对话功能，能够根据配置的模板，回复包含上下文的对话。

缺点：只能匹配到模板中的话术，同一个意图，不同的话术，需要配置多个不同的模板，维护成本高。

改进：文本相似度替模板匹配。

2. 什么是数据驱动？数据驱动如何在这个程序里体现？

数据驱动：程序根据输入的数据的特征，匹配相关的内容，生成对应的结果。
数据驱动在对话中的体现：对话程序通过匹配输入话术，匹配到对应的模板，生成对应响应。

3. 数据驱动与AI的关系

AI程序提现了数据驱动的思想。如果AI看做一个函数$y=f(x)$，拟合不同的数据特征，能够根据不同的输入数据$x$，输出响应相匹配的输出$y$。